# Predicting Numbers of House Sales in England and Wales

## Random Forest Regressor

### Preamble

In [73]:
# Configure libraries

%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import mean_squared_error
import random

In [74]:
# Funtion for cross-validation over a grid of parameters

def cv_optimize(clf, parameters, X, y, n_jobs=1, n_folds=5, score_func=None, verbose=0):
    if score_func:
        gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds, n_jobs=n_jobs, scoring=score_func, verbose=verbose)
    else:
        gs = GridSearchCV(clf, param_grid=parameters, n_jobs=n_jobs, cv=n_folds, verbose=verbose)
    gs.fit(X, y)
    print("BEST"), gs.best_params_, gs.best_score_, gs.grid_scores_, gs.scorer_
    print("Best score: "), gs.best_score_
    best = gs.best_estimator_
    return best

### Reading in the data

In [75]:
# Each line is of the format:
df_house = pd.read_csv("df_house data.csv")
names = ["year", "freq", "day", "month", "weekday", "week", "weekend"]
df_house = df_house[names]
print(df_house.shape)

(8548, 7)


### Define train and test sets

In [76]:
itrain, itest = train_test_split(range(df_house.shape[0]), train_size=0.8)
mask = np.ones(df_house.shape[0], dtype = 'int')
mask[itrain] = 1
mask[itest] = 0
mask = (mask == 1)
mask[:10]

array([ True, False,  True, False,  True,  True,  True,  True,  True,  True], dtype=bool)

### Final preparation for machine learning

In [77]:
# Split off the features
Xnames = ["year", "day", "month", "weekday", 
          "week", "weekend"]
X = df_house[Xnames]

# Split off the target (which will be the logarithm of the number of house sales (+1))
y = np.log10(df_house['freq']+1)

### Get train and test sets

In [78]:
Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
n_samples = Xtrain.shape[0]
n_features = Xtrain.shape[1]
print(Xtrain.shape)
max_samples = 1000000
if Xtrain.shape[0] > max_samples:
    rows = random.sample(Xtrain.index, max_samples)
    Xtrain = Xtrain.ix[rows]
    ytrain = ytrain.ix[rows]
print(Xtrain.shape)
Xtrain.head()

(6838, 6)
(6838, 6)


,year,day,month,weekday,week,weekend
0,1995,1,1,0,52,1
2,1995,3,1,2,1,0
4,1995,5,1,4,1,0
5,1995,6,1,5,1,0
6,1995,7,1,6,1,1


### Random Forest Regression

In [79]:
# Create a Random Forest Regression estimator
estimator = RandomForestRegressor(n_estimators = 20, n_jobs = -1)

The step below takes about 25 mins on my laptop for 300,000 records and up to 100 estimators.

In [80]:
%%time
# Define a grid of parameters over which to optimize the random forest
# We will figure out which number of trees is optimal
parameters = {"n_estimators": [50],
              "max_features": ["auto"], # ["auto","sqrt","log2"]
              "max_depth": [50]}
best = cv_optimize(estimator, parameters, Xtrain, ytrain, n_folds = 5, score_func='mean_squared_error', verbose = 3)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] max_depth=50, max_features=auto, n_estimators=50 ................


C:\Python36\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  max_depth=50, max_features=auto, n_estimators=50, score=-0.058668 -   0.5s
[CV] max_depth=50, max_features=auto, n_estimators=50 ................


C:\Python36\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s


[CV]  max_depth=50, max_features=auto, n_estimators=50, score=-0.055730 -   0.5s
[CV] max_depth=50, max_features=auto, n_estimators=50 ................


...

[CV]  max_depth=50, max_features=auto, n_estimators=50, score=-0.083154 -   0.6s
[CV] max_depth=50, max_features=auto, n_estimators=50 ................


C:\Python36\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV]  max_depth=50, max_features=auto, n_estimators=50, score=-0.085145 -   0.6s
[CV] max_depth=50, max_features=auto, n_estimators=50 ................


C:\Python36\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.4s finished


[CV]  max_depth=50, max_features=auto, n_estimators=50, score=-0.102242 -   0.7s
BEST
Best score: 
Wall time: 4.41 s


### Evaluate the results

In [81]:
# Fit the best Random Forest and calculate R^2 values for training and test sets
reg = best.fit(Xtrain, ytrain)
training_accuracy = reg.score(Xtrain, ytrain)
test_accuracy = reg.score(Xtest, ytest)
print("############# based on standard predict ################")
print("R^2 on training data: %0.4f" % (training_accuracy))
print("R^2 on test data:     %0.4f" % (test_accuracy))

############# based on standard predict ################
R^2 on training data: 0.9913
R^2 on test data:     0.9481


In [82]:
# Show some of the predictions vs. the real number of house sales
np.round(np.power(10,np.column_stack((reg.predict(Xtest),ytest))) - 1,decimals=0).astype(int)

array([[ 141,   68],
       [1115, 1167],
       [1150, 1021],
       ..., 
       [ 446,  837],
       [ 719,  621],
       [ 205,   84]])

In [83]:
# Calculate the Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(reg.predict(Xtest),ytest))
print("RMSE = %0.3f (this is in log-space!)" % rmse)
print("So two thirds of the records would be a factor of less than %0.2f away from the real value." % np.power(10,rmse))

RMSE = 0.233 (this is in log-space!)
So two thirds of the records would be a factor of less than 1.71 away from the real value.


In [84]:
# What are the most important features?
import operator
dict_feat_imp = dict(zip(list(X.columns.values),reg.feature_importances_))

sorted_features = sorted(dict_feat_imp.items(), key=operator.itemgetter(1), reverse=True)
sorted_features

[('weekend', 0.79909170465762958),
 ('week', 0.056661961990846638),
 ('weekday', 0.051773187657289686),
 ('year', 0.04389626236160981),
 ('day', 0.04054588829813132),
 ('month', 0.0080309950344931574)]